In [1]:
!nvidia-smi

Thu Feb  6 16:57:58 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   49C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
!pip install -U transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [6]:
!pip install -U accelerate datasets transformers sacrebleu rouge_score py7zr -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.6/336.6 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.3 MB/s eta 0:00:00


In [7]:
import torch
from transformers import pipeline, set_seed, AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
import pandas as pd

import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Basic HF model infarence

In [ ]:
# from transformers import AutoTokenizer, PegasusForConditionalGeneration
# model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")
# tokenizer = AutoTokenizer.from_pretrained("google/pegasus-xsum")

# ARTICLE_TO_SUMMARIZE = "PG&E stated it scheduled the blackouts in response to forecasts for high winds amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were scheduled to be affected by the shutoffs which were expected to last through Wednesday."\

# inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors='pt', truncation=True)
# # summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=5, early_stopping=True)
# summary_ids = model.generate(inputs['input_ids'])
# tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


# Fine Tuning

In [9]:
model_ckpt = 'google/pegasus-cnn_dailymail'

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [11]:
!wget https://github.com/krishnaik06/datasets/raw/refs/heads/main/summarizer-data.zip
!unzip summarizer-data.zip

--2025-02-06 17:05:11--  https://github.com/krishnaik06/datasets/raw/refs/heads/main/summarizer-data.zip
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/krishnaik06/datasets/refs/heads/main/summarizer-data.zip [following]
--2025-02-06 17:05:12--  https://raw.githubusercontent.com/krishnaik06/datasets/refs/heads/main/summarizer-data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7903594 (7.5M) [application/zip]
Saving to: ‘summarizer-data.zip’

summarizer-data.zip 100%[===================>]   7.54M  --.-KB/s    in 0.04s   

2025-02-06 17:05:12 (168 MB/s) - ‘summarizer-data.zip’ saved [790359

In [12]:
dataset_samsum = load_from_disk('./samsum_dataset')
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [13]:
split_lengths = [len(dataset_samsum[split]) for split in dataset_samsum.keys()]

print(f"Train: {split_lengths[0]}, Validation: {split_lengths[1]}, Test: {split_lengths[2]}")
print(f"Feature Names: {dataset_samsum['train'].column_names}")

print(f"Dialogue: \n{dataset_samsum['train'][1]['dialogue']}")
print(f"Summary: \n{dataset_samsum['train'][1]['summary']}")

Train: 14732, Validation: 819, Test: 818
Feature Names: ['id', 'dialogue', 'summary']
Dialogue: 
Olivia: Who are you voting for in this election? 
Oliver: Liberals as always.
Olivia: Me too!!
Oliver: Great
Summary: 
Olivia and Olivier are voting for liberals in this election. 


### Expected input from for Seq2Seq model fine-tuning

```
{
    'input_ids' : [123, 456, 789, ...]  # token ids for the dialogue (input features)
    'attention_mask': [1, 1, 1, 1, ...] # attention mask for the input
    'labels': [321, 654, 987, ...]      # Token ids for the summary (tagets)
}
```

In [14]:
def conver_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['dialogue'], truncation=True, padding='max_length', max_length=1024)

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], truncation=True, padding='max_length', max_length=128)

    encodings = {
        'input_ids': input_encodings.input_ids,
        'attention_mask': input_encodings.attention_mask,
        'labels': target_encodings.input_ids
    }
    return encodings

In [15]:
dataset_samsum_pt = dataset_samsum.map(conver_examples_to_features, batched=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [16]:
dataset_samsum_pt['train']

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [17]:
## training the model

from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [20]:
# from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='pegasus_samsum',
    num_train_epochs=1,
    warmup_steps=500,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    logging_steps = 10,
    logging_dir='logs',
    evaluation_strategy='steps',
    eval_steps=500,
    save_steps=999,
    gradient_accumulation_steps=16,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [22]:
trainer = Trainer(
    model=model_pegasus,
    args= training_args,
    processing_class=tokenizer,
    data_collator=seq2seq_data_collator,
    # train_dataset=dataset_samsum_pt['train'],
    train_dataset=dataset_samsum_pt['test'],
    eval_dataset=dataset_samsum_pt['validation'],
)

In [23]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sadhin to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss,Validation Loss


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=51, training_loss=10.526268996444403, metrics={'train_runtime': 833.5639, 'train_samples_per_second': 0.983, 'train_steps_per_second': 0.061, 'total_flos': 2357802961993728.0, 'train_loss': 10.526268996444403, 'epoch': 0.9963369963369964})

In [32]:
import gc
gc.collect()
torch.cuda.empty_cache()

## Evaluation

In [25]:
def generate_batch_sized_chunks(list_of_elements, batch_size):

    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i:i+batch_size]

def calculate_metric_on_test_data(dataset, metric, model, tokenizers,
                                batch_size=16, device=device,column_text='article', column_summary="summary"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    summary_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, summary_batch in tqdm(zip(article_batches, summary_batches), total=len(article_batches)):
        # inputs = tokenizers(article_batch, max_length=1024, return_tensors='pt', truncation=True, padding=True).to(device)
        # outputs = model.generate(**inputs)
        # predictions = tokenizers.batch_decode(outputs, skip_special_tokens=True)
        # metric.add_batch(predictions=predictions, references=summary_batch)

        inputs = tokenizer(article_batch, max_length=1024, truncation=True, padding=True, return_tensors='pt').to(device)

        summaries = model.generate(input_ids = inputs["input_ids"].to(device),
                                   attention_mask=inputs['attention_mask'].to(device),
                                   length_penalty=0.8, num_beams=9, max_length=128)
        """ parameter for length penalty to avoid the too long sequence to generate the model """

        decode_summaries = [tokenizer.decode(s, skip_special_tokens=True,clean_up_tokenization_spaces=True)
                            for s in summaries]
        decode_summaries = [d.replace('', ' ') for d in decode_summaries]
        metric.add_batch(predictions=decode_summaries, references=summary_batch)

    score = metric.compute()
    return score

In [26]:
!pip install -U evaluate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.8 MB/s eta 0:00:00


In [27]:
import evaluate

rouge_metric = evaluate.load('rouge')
rouge_names = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']

In [33]:
score = calculate_metric_on_test_data(
    dataset_samsum['train'][0:5],
    rouge_metric, model_pegasus, tokenizer, batch_size=16, device=device, column_text='dialogue', column_summary='summary')

rouge_dict = {rn: score[rn] for rn in rouge_names}
import pandas as pd
pd.DataFrame(rouge_dict, index=['pegasus'])

100%|██████████| 1/1 [00:10<00:00, 10.23s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.001262,0.0,0.001262,0.001262


In [34]:
torch.cuda.empty_cache()

In [35]:
model_pegasus.save_pretrained('pegasus-samsum-finetuned')
tokenizer.save_pretrained('pegasus-samsum-finetuned')

('pegasus-samsum-finetuned/tokenizer_config.json',
 'pegasus-samsum-finetuned/special_tokens_map.json',
 'pegasus-samsum-finetuned/spiece.model',
 'pegasus-samsum-finetuned/added_tokens.json',
 'pegasus-samsum-finetuned/tokenizer.json')

In [36]:
torch.cuda.empty_cache()

In [37]:
tokenizer = AutoTokenizer.from_pretrained('pegasus-samsum-finetuned')

gen_kwargs = {'length_penalty': 0.8, 'num_beams': 9, 'max_length': 128}


pipe = pipeline('summarization', model='./pegasus-samsum-finetuned', tokenizer=tokenizer )

Device set to use cuda:0


In [39]:
sample_text = dataset_samsum['test'][5]['dialogue']
reference = dataset_samsum['test'][5]['summary']

print("Dialogue: ", sample_text)
print("Summary: ", reference)

out = pipe(sample_text, **gen_kwargs)[0]['summary_text']
print("Model Summary: ", out)

Dialogue:  Benjamin: Hey guys, what are we doing with the keys today?
Hilary: I've got them. Whoever wants them can meet me at lunchtime or after
Elliot: I'm ok. We're meeting for the drinks in the evening anyway and I guess we'll be going back to the apartment together?
Hilary: Yeah, I guess so
Daniel: I'm with Hilary atm and won't let go of her for the rest of the day, so any option you guys choose is good for me
Benjamin: Hmm I might actually pass by at lunchtime, take the keys and go take a nap. I'm sooo tired after yesterday
Hilary: Sounds good. We'll be having lunch with some French people (the ones who work on the history of food in colonial Mexico - I already see you yawning your head off)
Benjamin: YAAAAWN 🙊 Where and where are you meeting?
Hilary: So I'm meeting them at the entrance to the conference hall at 2 pm and then we'll head to this place called La Cantina. Italian cuisine, which is quite funny, but that's what they've chosen
Benjamin: Interesting 😱 To be honest, Hila

# Finish